In [1]:
# !pip install transformers

In [ ]:
# import os
# # import Google Drive 套件
# from google.colab import drive
# # 將自己的雲端硬碟掛載上去
# drive.mount('/content/gdrive')

# os.chdir('./gdrive/MyDrive/成大/justLearn/Jigsaw Rate Severity of Toxic Comments/')      # 檔案目錄

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
from transformers import AutoTokenizer, AutoModel
import pickle
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

In [2]:
# 留下前510字的 function
def stringProcess(data):
    input_ids, token_type_ids, attention_mask = data['input_ids'], data['token_type_ids'], data['attention_mask']
    data['input_ids'] = data['input_ids'][:, :512]
    data['input_ids'][:, -1] = 102
    data['token_type_ids'] = data['token_type_ids'][:, :512]
    data['attention_mask'] = data['attention_mask'][:, :512]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# 載入並宣告bert模型
tokenizer = AutoTokenizer.from_pretrained('./model/tokenizer/')
model = AutoModel.from_pretrained("./model/bert_model/").to(device)

In [4]:
# tokenizer.save_pretrained("./model/tokenizer/")
# model.save_pretrained('./model/bert')

In [ ]:
# load data
id = 1
path = f'./data/trainData/training/data2/data2_{id}.pickle'
with open(path, 'rb') as f:
    data = pickle.load(f)
comment_text = data['comment_text']

path = f'./data/trainData/training/data2_{id}_embedding.pickle'
embedding_vec = []
for txt in tqdm(comment_text):
    with torch.no_grad():
        inputs = tokenizer(txt, return_tensors="pt")
        inputs = inputs.to(device)
        stringProcess(inputs)
        outputs = model(**inputs)
        embedding_vec.append(outputs[1].cpu().numpy().astype(float))
    # break
data['embedding_vec'] = np.concatenate(embedding_vec, axis=0)
with open(path, 'wb') as f:
    pickle.dump(data, f)

100%|██████████| 100000/100000 [22:39<00:00, 73.56it/s]


In [ ]:
# data split
path = './data/trainData/training/data2.pickle'
with open(path, 'rb') as f:
    data = pickle.load(f)

l = len(data['score']) // 100000
for i in range(l):
  a, b = i * 100000, (i+1) * 100000
  d = {'comment_text':data['comment_text'][a:b], 'score':data['score'][a:b]}
  with open(f'./data/trainData/training/data2/data2_{i+1}.pickle', 'wb') as f:
      pickle.dump(d, f)